In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import numpy as np
import pandas as pd
import tarfile
import matplotlib.pyplot as plt
import scipy as sp
import csv
import random
import math
import operator
import os
from collections import Counter
import nltk
from sklearn.model_selection import train_test_split


In [ ]:


amazon_dataframe = pd.read_csv("/kaggle/input/sentiment-labelled-sentences-data-set/sentiment labelled sentences/sentiment labelled sentences/amazon_cells_labelled.txt", 
                        delimiter='\t', names=['review', 'sentiment'])

yelp_dataframe = pd.read_csv("/kaggle/input/sentiment-labelled-sentences-data-set/sentiment labelled sentences/sentiment labelled sentences/yelp_labelled.txt", 
                        delimiter='\t', names=['review', 'sentiment'])

df_train = pd.DataFrame()
df_dev = pd.DataFrame()
df_test = pd.DataFrame()

frame = [amazon_dataframe, yelp_dataframe]
final_dataframe = pd.concat(frame) # combining the data from amazon and yelp inorder to increase the accuracy ( just a try.)

final_dataframe.reset_index(drop='True', inplace=True)

split_1 = int(0.8 * len(final_dataframe))
split_2 = int(0.9 * len(final_dataframe))
df_train = final_dataframe[:split_1]
df_dev = final_dataframe[split_1:split_2]
df_test = final_dataframe[split_2:]

Combining the data increased the accuracy from 67% to 69.5%

In [ ]:


df_train_len = len(df_train)
print('total train records:', df_train_len)

df_train_pos_len = len(df_train[df_train['sentiment'] == 1])
df_train_neg_len = len(df_train[df_train['sentiment'] == 0])
prob_pos_train = df_train_pos_len / df_train_len
prob_neg_train = df_train_neg_len / df_train_len

print ('positive records:', df_train_pos_len)
print ('negative records:', df_train_neg_len)

print ('prob positive records:', prob_pos_train)
print ('prob negative records:', prob_neg_train)

print()
print('total dev records:', len(df_dev))
print('total test records:', len(df_test))

In [ ]:

df_train.groupby('sentiment').count()

In [ ]:

df_dev.groupby('sentiment').count()

In [ ]:

df_test.groupby('sentiment').count()

In [ ]:


np.random.seed(0)
df_train = df_train.reindex(np.random.permutation(df_train.index))
df_train.to_csv('movie_data_train.csv', index=False, encoding = 'utf-8')

np.random.seed(0)
df_dev = df_dev.reindex(np.random.permutation(df_dev.index))
df_dev.to_csv('movie_data_dev.csv', index=False, encoding = 'utf-8')

np.random.seed(0)
df_test = df_test.reindex(np.random.permutation(df_test.index))
df_test.to_csv('movie_data_test.csv', index=False, encoding = 'utf-8')

In [ ]:
# Print train data
df_train = pd.read_csv('movie_data_train.csv', encoding = 'utf-8')
df_train.head(5)

In [ ]:

df_dev = pd.read_csv('movie_data_dev.csv', encoding = 'utf-8')
df_dev.head(5)

In [ ]:

df_test = pd.read_csv('movie_data_test.csv', encoding = 'utf-8')
df_test.head(5)

In [ ]:


doc_len = len(df_train)
print(doc_len)

Cleaning the data 


In [ ]:


df_train.columns = df_train.columns.str.strip()         
df_train.columns = df_train.columns.str.replace(r"[^a-zA-Z\d\_]+", "")    
df_train.columns = df_train.columns.str.replace(r"[^a-zA-Z\d\_]+", "")

df_dev.columns = df_dev.columns.str.strip()         
df_dev.columns = df_dev.columns.str.replace(r"[^a-zA-Z\d\_]+", "")    
df_dev.columns = df_dev.columns.str.replace(r"[^a-zA-Z\d\_]+", "")

df_test.columns = df_test.columns.str.strip()         
df_test.columns = df_test.columns.str.replace(r"[^a-zA-Z\d\_]+", "")    
df_test.columns = df_test.columns.str.replace(r"[^a-zA-Z\d\_]+", "")

df_train = df_train.replace([";",":","=","\+","<", ">", "\?", "!", "\\\\", "@", "#", "$", "\*", "%", ",", "\.", "\(", "\)", "\[", "\]", "\{", "\}", "\"", "/br"], "", regex = True)
df_dev = df_dev.replace([";",":","=","\+","<", ">", "\?", "!", "\\\\", "@", "#", "$", "\*", "%", ",", "\.", "\(", "\)", "\[", "\]", "\{", "\}", "\"", "/br"], "", regex = True)
df_test = df_test.replace([";",":","=","\+","<", ">", "\?", "!", "\\\\", "@", "#", "$", "\*", "%", ",", "\.", "\(", "\)", "\[", "\]", "\{", "\}", "\"", "/br"], "", regex = True)

df_train = df_train.replace(["' ", " '"], " ", regex = True)
df_dev = df_dev.replace(["' ", " '"], " ", regex = True)
df_test = df_test.replace(["' ", " '"], " ", regex = True)

Below we Build a vocabulary as list. 

In [ ]:


wordfreq = dict()
wordfreq_pos = dict()
wordfreq_neg = dict()
for ind in df_train.index:
    review_set = set(df_train['review'][ind].lower().split())
    for word in review_set:
        if word in wordfreq:
            wordfreq[word] += 1
        else:
            wordfreq[word] = 1
        
        if df_train['sentiment'][ind] == 1:
            if word in wordfreq_pos:
                wordfreq_pos[word] += 1
            else:
                wordfreq_pos[word] = 1
        else:
            if word in wordfreq_neg:
                wordfreq_neg[word] += 1
            else:
                wordfreq_neg[word] = 1

we omit rare words for example if the occurrence is less than five times

In [ ]:


final_vocab = dict()
final_vocab_pos = dict()
final_vocab_neg = dict()
for word in wordfreq:
    if wordfreq[word] > 5:
        final_vocab[word] = wordfreq[word]
    if word in wordfreq_pos:
        if wordfreq_pos[word] > 5:
            final_vocab_pos[word] = wordfreq_pos[word]
    if word in wordfreq_neg:
        if wordfreq_neg[word] > 5:
            final_vocab_neg[word] = wordfreq_neg[word]

In [ ]:

print("Total Vocab:",len(final_vocab))

**Calculate the following probability**

a)Probability of the occurrence
b)Conditional probability based on the sentiment
****

In [ ]:

prob_word = dict()
prob_word_g_pos = dict()
prob_word_g_neg = dict()

for word in final_vocab:
    prob_word[word] = final_vocab[word] / doc_len
    if word in final_vocab_pos:
        prob_word_g_pos[word] = final_vocab_pos[word] / df_train_pos_len
        
    if word in final_vocab_neg:
        prob_word_g_neg[word] = final_vocab_neg[word] / df_train_neg_len

**Calculate accuracy using dev dataset** 

* Conduct five fold cross validation

In [ ]:

accuracy_normal = []
df_dev_arr = np.array_split(df_dev, 5)
ctr = 0

print("Accuracy using 5-Fold Cross Validation:")

for df in df_dev_arr:
    count = 0
    ctr += 1
    predicted_sentiments = []
    prob_pos_g_wir = dict()
    prob_neg_g_wir = dict()
    
    for ind in df.index:
        numPos = 0.00
        numNeg = 0.00
        
        review_set = set(df['review'][ind].lower().split())
        for word in review_set:
            if word in prob_word:
                if word not in prob_word_g_pos:
                    numNeg = 0
                elif word not in prob_word_g_neg:
                    numPos = 0
                else:
                    numPos = numPos + math.log(prob_word_g_pos[word])
                    numNeg = numNeg + math.log(prob_word_g_neg[word])
                            
        prob_pos_g_wir[ind] = pow(math.e, numPos) * prob_pos_train
        prob_neg_g_wir[ind] = pow(math.e, numNeg) * prob_neg_train
                            
        if(prob_pos_g_wir[ind] < prob_neg_g_wir[ind]):
            predicted_sentiments.append(0)
        else:
            predicted_sentiments.append(1)
                                    
    df['prediction'] = predicted_sentiments
                                                                        
    for ind in df.index:
        if df['sentiment'][ind] == df['prediction'][ind]:
            count += 1
                                            
    accuracy = count / len(df)
    accuracy_normal.append(accuracy)
    print (ctr,": Accuracy df_dev:",accuracy*100,"%")

**Compare the effect of Smoothing**

In [ ]:

prob_word_g_pos_smooth = dict()
prob_word_g_neg_smooth = dict()

for word in final_vocab:
    if word in final_vocab_pos:
        prob_word_g_pos_smooth[word] = (final_vocab_pos[word]+1) / (df_train_pos_len + len(final_vocab))
        
    if word in final_vocab_neg:
        prob_word_g_neg_smooth[word] = (final_vocab_neg[word]+1) / (df_train_neg_len + len(final_vocab))

In [ ]:


accuracy_smooth = []
df_dev_arr = np.array_split(df_dev, 5)
ctr = 0

print("Accuracy after Smoothing using 5-Fold Cross Validation:")

for df in df_dev_arr:
    count = 0
    ctr += 1
    predicted_sentiments = []
    prob_pos_g_wir = dict()
    prob_neg_g_wir = dict()
    
    for ind in df.index:
        numPos = 0.00
        numNeg = 0.00
        
        review_set = set(df['review'][ind].lower().split())
        for word in review_set:
            if word in prob_word:
                if word not in prob_word_g_pos:
                    numNeg = 0
                elif word not in prob_word_g_neg:
                    numPos = 0
                else:
                    numPos = numPos + math.log(prob_word_g_pos_smooth[word])
                    numNeg = numNeg + math.log(prob_word_g_neg_smooth[word])
                            
        prob_pos_g_wir[ind] = pow(math.e, numPos) * prob_pos_train
        prob_neg_g_wir[ind] = pow(math.e, numNeg) * prob_neg_train
                            
        if(prob_pos_g_wir[ind] < prob_neg_g_wir[ind]):
            predicted_sentiments.append(0)
        else:
            predicted_sentiments.append(1)
                                    
    df['prediction'] = predicted_sentiments
                                                                        
    for ind in df.index:
        if df['sentiment'][ind] == df['prediction'][ind]:
            count += 1
                                            
    accuracy = count / len(df)
    accuracy_smooth.append(accuracy)
    print (ctr,": Accuracy df_dev:",accuracy*100,"%")

In [ ]:


betterNormal = 0
betterSmoothing = 0

for i in range(len(accuracy_normal)):
    if accuracy_normal[i] > accuracy_smooth[i]:
        betterNormal += 1
    else:
        betterSmoothing +=1

if(betterNormal > betterSmoothing):
    print("For the given dev dataset, accuracy is better without smoothing")
else:
    print("For the given dev dataset, accuracy is better with smoothing")

**Derive Top 10 words that predicts positive and negative class
P[Positive| word] 
**

In [ ]:


prob_pos_given_word = dict()
prob_neg_given_word = dict()

for word in final_vocab:
    if word in final_vocab_pos:
        prob_pos_given_word[word] = (prob_word_g_pos[word] * prob_pos_train) / prob_word[word]
    if word in final_vocab_neg:
        prob_neg_given_word[word] = (prob_word_g_neg[word] * prob_neg_train) / prob_word[word]

**Positive Words**

In [ ]:

print("Top 10 words predicting positive class:")
prob_pos_given_word = sorted(prob_pos_given_word.items(), key=operator.itemgetter(1), reverse=True)

prob_pos_given_word[:10]

**Negative Words**

In [ ]:

print("Top 10 words predicting negative class:")
prob_neg_given_word = sorted(prob_neg_given_word.items(), key=operator.itemgetter(1), reverse=True)
prob_neg_given_word[:10]

**calculate the final accuracy using the hyperparameters found from the above step**

In [ ]:

predicted_sentiments = []
prob_pos_g_wir = dict()
prob_neg_g_wir = dict()

if(betterNormal < betterSmoothing):
    prob_word_g_pos = prob_word_g_pos_smooth

for ind in df_test.index:
    numPos = 0.00
    numNeg = 0.00
    
    review_set = set(df_test['review'][ind].lower().split())
    for word in review_set:
        if word in prob_word:
            if word not in prob_word_g_pos:
                numNeg = 0
            elif word not in prob_word_g_neg:
                numPos = 0
            else:
                numPos = numPos + math.log(prob_word_g_pos[word])
                numNeg = numNeg + math.log(prob_word_g_neg[word])
    
    prob_pos_g_wir[ind] = pow(math.e, numPos) * prob_pos_train
    prob_neg_g_wir[ind] = pow(math.e, numNeg) * prob_neg_train
    
    if(prob_pos_g_wir[ind] < prob_neg_g_wir[ind]):
        predicted_sentiments.append(0)
    else:
        predicted_sentiments.append(1)
        
df_test['prediction'] = predicted_sentiments

count = 0

for ind in df_test.index:
    if df_test['sentiment'][ind] == df_test['prediction'][ind]:
        count += 1
        
accuracy = count / len(df_test)
print ("Accuracy df_test:",accuracy*100,"%")

The accuracy is found to be 69.5%

**References**

https://www.dataquest.io/blog/naive-bayes-tutorial/
https://stackoverflow.com/questions/51085553/scikit-learn-5-fold-cross-validation-train-test-split
https://stackoverflow.com/questions/20618804/how-to-smooth-a-curve-in-the-right-way
https://stackabuse.com/the-naive-bayes-algorithm-in-python-with-scikit-learn/
